In [1]:
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import xarray as xr
import pandas as pd
import scipy
from datetime import datetime, timedelta
import time
import seaborn
import matplotlib.dates as mdates
import bottleneck as bn
import PyCO2SYS as pyco2
import gsw
import math
import netCDF4 as nc
import requests
import statsmodels.api as sm

# import chemO_funcs #import o2sat
from importlib import reload
import warnings
warnings.filterwarnings('ignore')
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
# %run ./chemO_funcs.py

# Present data

- open dataset
- take annual means
- rename time dimension to `t`

In [2]:
ds_present = xr.open_mfdataset('/Users/akbaskind/Desktop/YoE/GFDLESM4historical_1995_2015.nc')

In [5]:
dd_present_ann = ds_present.groupby('time.year').mean(dim="time")

In [13]:
dd_present_ann = dd_present_ann.rename({'year': 't'})

# Future data

- open dataset
- take annual means
- merge the two future datasets
- rename time dimension to `t`

In [3]:
ds1 = xr.open_mfdataset('/Users/akbaskind/Desktop/YoE/GFDLESM4future_2015_2035.nc')
ds2 = xr.open_mfdataset('/Users/akbaskind/Desktop/YoE/GFDLESM4_2035_2060.nc')
ds_future = xr.concat([ds1, ds2], dim="time")

In [6]:
dd_future_ann = ds_future.groupby('time.year').mean(dim="time")

In [11]:
dd_future_ann = dd_future_ann.rename({'year': 't'})

# Past data

- data already in decadal means
- rename `ph` to `phos` to make consistent with other datasets

In [4]:
ds_past = xr.open_mfdataset('/Users/akbaskind/Desktop/YoE/GFDLESM4historical_1850_2000_means.nc')

In [19]:
ds_past = ds_past.rename({'ph': 'phos'})
ds_past

<xarray.Dataset>
Dimensions:         (t: 15, y: 180, x: 360)
Coordinates:
  * t               (t) int64 1850 1860 1870 1880 1890 ... 1960 1970 1980 1990
  * y               (y) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * x               (x) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    lat_bounds      (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
    lon_bounds      (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
    lon             (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
    lat             (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
    lon_verticies   (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
    lat_verticies   (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
    member_id       object ...
    dcpp_init_year  float64 ...
Data variables:
    fCO2            (t, y, x) float64 dask.array<chunksize=(15, 180, 360), meta=np.ndarray>
    OmegaAr         (t, y, x) float64 dask.array<chunksize=(15, 180, 360), meta=np.ndarray>
    phos            (t, y, x) float64 dask.array<chunksize=(15, 180, 360), meta=np.ndarray>

# Combine past, present, and future and save

In [20]:
dd = xr.concat([ds_past, dd_present_ann[['fCO2', 'phos', 'OmegaAr']], dd_future_ann[['fCO2', 'phos', 'OmegaAr']]], dim="t")

In [23]:
dd.to_netcdf('/Users/akbaskind/Desktop/YoE/GFDLESM4_1850_2060_decadalandannual.nc')